In [1]:
import os

### openBLAS will not work properly in jupyter notebook
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import pandas as pd
import numpy as np
from scipy import stats as ss
import snapatac2 as snap
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors

## Process snATAC: read, merge all snap file

In [37]:
meta = pd.read_csv('hba.whole.cell2anno.txt', sep = '\t')

In [7]:
afile = [name + "_processed.h5ad" for name in np.unique(meta['sample'])]
adatas = ["/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/snap/" + name for name in afile]
data = snap.AnnDataSet(
    adatas = [(name, adata) for name, adata in zip(np.unique(meta['sample']), adatas)],
    filename = "/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_merged.h5ads"
)

### make sure barcode is unique
unique_cell_ids = [sa + ':' + bc for sa, bc in zip(data.obs['sample'], data.obs_names)]
data.obs_names = unique_cell_ids

Unable to create stacked arrays for these keys: 


In [5]:
tdata = data.to_adata()

### processing...
### feature selection
snap.pp.select_features(tdata, n_features = 100000,
                        blacklist = "/projects/ps-renlab/y2xie/projects/genome_ref/hg38_CnR_blacklist.bed")
snap.tl.spectral(tdata, weighted_by_sd = True, random_state = 921)
if len(tdata.obs_names) >= 500000:
    k = 50
elif (len(tdata.obs_names) >= 200000 & len(tdata.obs_names) < 500000):
    k = 25
else:
    k = 15

print("performing leiden clustering...")
snap.pp.knn(tdata, n_neighbors = k, random_state = 921)
snap.tl.umap(tdata, random_state = 921)
snap.tl.leiden(tdata, resolution = 0.5, objective_function = 'RBConfiguration',
               use_leidenalg = True, weighted = True)
tdata.write("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_merged.h5ad")

2024-04-02 14:05:08 - INFO - Selected 99923 features.


performing leiden clustering...


/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
... storing 'sample' as categorical
... storing 'leiden' as categorical


In [10]:
tdata.close()

## Process snATAC: subset by subclass

In [6]:
tdata = snap.read("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_merged.h5ad", backed=None)

In [38]:
### read meta with regions
meta2 = pd.read_csv('../../ref/yel_Science2023_supp/exp_metadata.txt', sep = '\t')
meta = meta.merge(meta2, left_on = 'sample', right_on = 'Sample')
meta.index = meta['sample'] + ":" + meta['barcode']
print(meta.shape)

(1136499, 17)


In [40]:
### write non-sorted neu% by region for comparsion
meta.to_csv('/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_merged.metadata.csv')

In [25]:
### subset data
tmeta = pd.read_csv('../../analysis/04.clustering/02.RNA/03.l3_clustering/MiniAtlas_RNA_merged_dual_filt_clean_metadata_250520.txt', sep = '\t')
qry_region = np.unique(tmeta['region'])
meta = meta.loc[meta['Brain region'].isin(qry_region)]
print(meta.shape)

(477803, 17)


In [27]:
### 2k cell per subclass
cls = []
for f in np.unique(meta['subclass']):
    tmp = meta.loc[meta['subclass'] == f].index.values
    if len(tmp) > 2000:
        cls.extend(np.random.choice(tmp, size=2000, replace=False))
    else:
        cls.extend(tmp)

print(len(cls))

cls = np.intersect1d(tdata.obs_names, cls)
print(len(cls))

47837
47833


In [30]:
## Subset anndata
ttdata = tdata[cls, :]
ttdata.obs[['cellclass', 'subclass']] = meta.loc[cls][['cellclass', 'subclass']]
ttdata.obs['donor'] = meta.loc[cls]['Donor'].astype(str)
## Rerun clustering
snap.pp.select_features(ttdata, n_features = 100000,
                        blacklist = "/projects/ps-renlab2/y2xie/ps-renlab/y2xie/projects/genome_ref/hg38_CnR_blacklist.bed")
snap.tl.spectral(ttdata, weighted_by_sd = True, random_state = 921)

### Batch correction
snap.pp.harmony(ttdata, batch = 'donor', use_rep='X_spectral')

k = 15
snap.pp.knn(ttdata, n_neighbors = k, random_state = 921, use_rep='X_spectral_harmony')
snap.tl.umap(ttdata, random_state = 921, use_rep='X_spectral_harmony')
snap.tl.leiden(ttdata, resolution = 0.5, objective_function = 'RBConfiguration',
               use_leidenalg = True, weighted = True)
ttdata.write("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_subsample_corrected_250601.h5ad")

/tmp/ipykernel_2057798/487080735.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ttdata.obs[['cellclass', 'subclass']] = meta.loc[cls][['cellclass', 'subclass']]
2025-06-01 11:46:58 - INFO - Selected 99936 features.
2025-06-01 11:48:28,906 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-06-01 11:48:28 - INFO - Computing initial centroids with sklearn.KMeans...
2025-06-01 11:48:33,518 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-06-01 11:48:33 - INFO - sklearn.KMeans initialization complete.
2025-06-01 11:48:33,677 - harmonypy - INFO - Iteration 1 of 10
2025-06-01 11:48:33 - INFO - Iteration 1 of 10
2025-06-01 11:48:40,303 - harmonypy - INFO - Iteration 2 of 10
2025-06-01 11:48:40 - INFO - Iteration 2 of 10
2025-06-01 11:48:44,356 - harmonypy - INFO - Iteration 3 of 10
2025-06-01 11:48:44 - INFO - Iteration 3 of 10
2025-06-01 11:48:47,728 - harmonypy - INFO - Converged after 3 

### Split neu / nonn

In [8]:
# ttdata = snap.read("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_subsample_corrected_250601.h5ad", backed = None)
qmeta = ttdata.obs
qcell = qmeta.loc[qmeta.cellclass == "NonN"].index
nonn = ttdata[qcell, :]

qcell = qmeta.loc[qmeta.cellclass != "NonN"].index
neu = ttdata[qcell, :]

In [9]:
snap.pp.select_features(nonn, n_features = 100000,
                        blacklist = "/projects/ps-renlab2/y2xie/ps-renlab/y2xie/projects/genome_ref/hg38_CnR_blacklist.bed")
snap.tl.spectral(nonn, weighted_by_sd = True, random_state = 921)

### Batch correction
snap.pp.harmony(nonn, batch = 'donor', use_rep='X_spectral')

k = 15
snap.pp.knn(nonn, n_neighbors = k, random_state = 921, use_rep='X_spectral_harmony')
snap.tl.umap(nonn, random_state = 921, use_rep='X_spectral_harmony')
snap.tl.leiden(nonn, resolution = 0.5, objective_function = 'RBConfiguration',
               use_leidenalg = True, weighted = True)

/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/snapatac2/preprocessing/_basic.py:976: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['count'] = count
2025-07-27 10:12:49 - INFO - Selected 99915 features.
2025-07-27 10:13:02,945 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-07-27 10:13:02 - INFO - Computing initial centroids with sklearn.KMeans...
2025-07-27 10:13:04,275 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-07-27 10:13:04 - INFO - sklearn.KMeans initialization complete.
2025-07-27 10:13:04,308 - harmonypy - INFO - Iteration 1 of 10
2025-07-27 10:13:04 - INFO - Iteration 1 of 10
2025-07-27 10:13:05,347 - harmonypy - INFO - Iteration 2 of 10
2025-07-27 10:13:05 - INFO - Iteration 2 of 10
2025-07-27 10:13:06,273 - harmonypy - INFO - Iteration 3 of 10
2025-07-27 10:13:06 - INFO - Iteration 3 of 10
2025-07-27 10:13:06,891 - harmonypy - INFO - Iteratio

In [10]:
snap.pp.select_features(neu, n_features = 100000,
                        blacklist = "/projects/ps-renlab2/y2xie/ps-renlab/y2xie/projects/genome_ref/hg38_CnR_blacklist.bed")
snap.tl.spectral(neu, weighted_by_sd = True, random_state = 921)

### Batch correction
snap.pp.harmony(neu, batch = 'donor', use_rep='X_spectral')

k = 15
snap.pp.knn(neu, n_neighbors = k, random_state = 921, use_rep='X_spectral_harmony')
snap.tl.umap(neu, random_state = 921, use_rep='X_spectral_harmony')
snap.tl.leiden(neu, resolution = 0.5, objective_function = 'RBConfiguration',
               use_leidenalg = True, weighted = True)

/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/snapatac2/preprocessing/_basic.py:976: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['count'] = count
2025-07-27 10:15:08 - INFO - Selected 99943 features.
2025-07-27 10:16:27,049 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-07-27 10:16:27 - INFO - Computing initial centroids with sklearn.KMeans...
2025-07-27 10:16:30,032 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-07-27 10:16:30 - INFO - sklearn.KMeans initialization complete.
2025-07-27 10:16:30,130 - harmonypy - INFO - Iteration 1 of 10
2025-07-27 10:16:30 - INFO - Iteration 1 of 10
2025-07-27 10:16:34,103 - harmonypy - INFO - Iteration 2 of 10
2025-07-27 10:16:34 - INFO - Iteration 2 of 10
2025-07-27 10:16:38,091 - harmonypy - INFO - Converged after 2 iterations
2025-07-27 10:16:38 - INFO - Converged after 2 iterations
/home/y2xie/miniconda3/envs/seu

In [13]:
nonn.write("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_subsample_corrected_250601.nonn.h5ad")
neu.write("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_subsample_corrected_250601.neu.h5ad")

... storing 'leiden' as categorical
... storing 'leiden' as categorical


## Generate gene activity matrix for integration

In [14]:
### subset each subclass 2k cells
gene_score = snap.read("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_merged_gene_score.h5ad", backed = None)
cls = np.intersect1d(gene_score.obs_names, ttdata.obs_names)
print(len(cls))

47833


In [33]:
qgene_score = gene_score[cls,:]
qgene_score.obs = ttdata.obs.loc[cls]
qgene_score.obsm = ttdata.obsm ### for Seurat V5 prediction
qgene_score.write("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_subsample_gene_score_250601.h5ad")

In [30]:
cls = np.intersect1d(nonn.obs_names, gene_score.obs_names)
qgene_score = gene_score[cls,:]
qgene_score.obs = nonn.obs.loc[cls]
qgene_score.obsm = nonn.obsm #['X_spectral'] 
qgene_score.write("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_subsample_gene_score_250601.nonn.h5ad")

In [31]:
cls = np.intersect1d(gene_score.obs_names, neu.obs_names)
qgene_score = gene_score[cls,:]
qgene_score.obs = neu.obs.loc[cls]
qgene_score.obsm = neu.obsm ### for Seurat V5 prediction
qgene_score.write("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_subsample_gene_score_250601.neu.h5ad")

## Generate matched cCRE matrix for downstream analysis

In [7]:
meta = pd.read_csv('/tscc/projects/ps-renlab2/yangli/projects/HBA/01.cluster/hba.barcode.sel.txt', sep = '\t')
meta.index = meta['sample'].astype(str) + ':' + meta['barcode'].astype(str)

meta2 = pd.read_csv('hba_merged.metadata.csv')
meta2.index = meta2['sample'].astype(str) + ':' + meta2['barcode'].astype(str)

In [6]:
data = snap.read_dataset("/projects/ps-renlab2/y2xie/projects/BICAN/ref/hba_ATAC/hba_merged.h5ads")
# atac_peaks = snap.pp.make_peak_matrix(data, peak_file = '/projects/ps-renlab2/y2xie/projects/BICAN/analysis/07.cCREs/MiniAtlas_matched_subclass.bed',# file = "",
#                          min_frag_size=10, max_frag_size=5000)
atac_peaks = snap.pp.make_peak_matrix(data, peak_file = '/projects/ps-renlab2/y2xie/projects/BICAN/analysis/07.cCREs/cCREs/hba.whole.union.peaks.bed',# file = "",
                         min_frag_size=10, max_frag_size=5000)

good_cells = np.intersect1d(atac_peaks.obs_names, meta.index)
atac_peaks = atac_peaks[good_cells,:]
atac_peaks

/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


View of AnnData object with n_obs × n_vars = 1136449 × 544735
    obs: 'sample'

In [14]:
atac_peaks.obs = meta2.loc[atac_peaks.obs_names]
atac_peaks.obs['n_fragment'] = np.sum(atac_peaks.X, axis = 1)

# atac_peaks.obs = meta.loc[atac_peaks.obs_names]
# atac_peaks.obs['n_fragment'] = meta2.loc[atac_peaks.obs_names]['UQ']

In [17]:
### Calculated on AST specifcially (aggregating ASCT and ASCNT)
atac_peaks.obs['subclass2'] = atac_peaks.obs['subclass']
atac_peaks.obs.loc[(atac_peaks.obs['subclass'].isin(['ASCT', 'ASCNT'])), 'subclass2'] = 'AST'
np.unique(atac_peaks.obs.subclass2)

array(['ACBGM', 'AMY', 'AST', 'BFEXA', 'BNGA', 'CBGA', 'CBGRC', 'CHO',
       'CNGA', 'CT', 'CTXGA', 'EC', 'ET', 'FOXP2', 'ITL23', 'ITL34',
       'ITL4', 'ITL45', 'ITL5', 'ITL6_1', 'ITL6_2', 'ITV1C', 'L6B',
       'LAMP5', 'LAMP5_LHX6', 'MBGA', 'MGC', 'MSN', 'NP', 'OGC', 'OPC',
       'PER', 'PRERC', 'PVALB', 'PV_ChCs', 'SIGA', 'SMC', 'SNCG', 'SST',
       'SST_CHODL', 'SUB', 'THMGA', 'VIP'], dtype=object)

In [31]:
atac_rpm = snap.tl.aggregate_X(atac_peaks, groupby = 'subclass')
total_frag = atac_peaks.obs.groupby('subclass')['n_fragment'].sum()
atac_rpm = pd.DataFrame(atac_rpm.X.T, index=atac_rpm.var_names, columns=atac_rpm.obs_names)

atac_cpm = (atac_rpm / total_frag) * 1e6
atac_cpm.to_csv('snATAC_subclass_cCRE.cpm.csv')

In [32]:
atac_rpm = snap.tl.aggregate_X(atac_peaks, groupby = 'subclass2')
total_frag = atac_peaks.obs.groupby('subclass2')['n_fragment'].sum()
atac_rpm = pd.DataFrame(atac_rpm.X.T, index=atac_rpm.var_names, columns=atac_rpm.obs_names)

atac_cpm = (atac_rpm / total_frag) * 1e6
atac_cpm.to_csv('snATAC_subclass2_cCRE.cpm.csv')

In [34]:
### also write celltype
atac_peaks.obs.loc[(atac_peaks.obs['celltype'] == 'D1Pu') | (atac_peaks.obs['celltype'] == 'D1CaB'), 'celltype'] = 'D1'
atac_peaks.obs.loc[(atac_peaks.obs['celltype'] == 'D2Pu') | (atac_peaks.obs['celltype'] == 'D2CaB'), 'celltype'] = 'D2'

atac_rpm = snap.tl.aggregate_X(atac_peaks, groupby = 'celltype')
total_frag = atac_peaks.obs.groupby('celltype')['n_fragment'].sum()
atac_rpm = pd.DataFrame(atac_rpm.X.T, index=atac_rpm.var_names, columns=atac_rpm.obs_names)

atac_cpm = (atac_rpm / total_frag) * 1e6
atac_cpm.to_csv('snATAC_celltype_cCRE.cpm.csv')

In [36]:
atac_peaks.write("snATAC_matched_MiniAtlas_cCRE.h5ad")

... storing 'sample' as categorical
... storing 'barcode' as categorical
... storing 'cellclass' as categorical
... storing 'subclass' as categorical
... storing 'celltype' as categorical
... storing 'Sample' as categorical
... storing 'Brain dissetion ID' as categorical
... storing 'Tn5 batch' as categorical
... storing 'Brain structure' as categorical
... storing 'Brain region' as categorical
... storing 'Library type' as categorical
... storing 'Sample quality' as categorical
... storing '# of nuclei passed QC' as categorical
... storing 'subclass2' as categorical


In [3]:
atac_peaks = snap.read("snATAC_matched_MiniAtlas_cCRE.h5ad", backed = None)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

mregion = []
for f in np.unique(atac_peaks.obs['subclass2']):
    ### select features to test
    file_path = f"../../analysis/07.cCREs/cCREs/subclass/{f}.bed"
    if os.path.exists(file_path):
        qfeatures = pd.read_csv(file_path, sep = '\t', names = ['chrom', 'start', 'end', 'id', 'score'])
        qfeatures['range'] = qfeatures['chrom'] + ":" + qfeatures['start'].astype(str) + "-" + qfeatures['end'].astype(str)
        fcell = atac_peaks.obs.loc[atac_peaks.obs['subclass2'] == f].index

        rcell = []
        for i in np.unique(atac_peaks.obs['subclass2']):
            if i != f: 
                cts = atac_peaks.obs.loc[atac_peaks.obs['subclass2'] == i].index
                if len(cts) > 1000:
                    cells = np.random.choice(cts, size=1000, replace=False)
                else:
                    cells = cts
                rcell.append(cells)      
        rcell = np.concatenate(rcell)
        fmarker = snap.tl.diff_test(atac_peaks, fcell, rcell, features = qfeatures['range'], min_pct = 0.01)
        if fmarker.shape[0] != 0:
            tmp = pd.DataFrame(fmarker, columns = fmarker.columns)
            tmp['celltype'] = f
            mregion.append(tmp)

2025-08-09 12:46:55 - INFO - Input contains 26737 features, now perform filtering with 'min_log_fc = 0.25' and 'min_pct = 0.01' ...
2025-08-09 12:46:58 - INFO - Testing 20391 features ...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20391/20391 [15:39<00:00, 21.70it/s]
2025-08-09 13:02:41 - INFO - Input contains 93699 features, now perform filtering with 'min_log_fc = 0.25' and 'min_pct = 0.01' ...
2025-08-09 13:02:50 - INFO - Testing 63900 features ...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63900/63900 [51:27<00:00, 20.70it/s]
2025-08-09 13:54:26 - INFO - Input contains 144727 features, now perform filtering with 'min_log_fc = 0.25' and 'min_pct = 0.01' ...
2025-08-09 13

 49%|█████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                | 52841/107015 [1:12:55<1:12:40, 12.42it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 59%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                               | 62703/107015 [1:26:10<57:42, 12.80it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_m

In [45]:
df = pd.concat(mregion, axis = 0)
np.unique(df.loc[df['adjusted p-value'] < 0.05].celltype, return_counts=True)

(array(['ACBGM', 'AMY', 'AST', 'BFEXA', 'BNGA', 'CBGA', 'CBGRC', 'CHO',
        'CNGA', 'CT', 'EC', 'ET', 'FOXP2', 'ITL23', 'ITL34', 'ITL4',
        'ITL45', 'ITL5', 'ITL6_1', 'ITL6_2', 'ITV1C', 'L6B', 'LAMP5',
        'LAMP5_LHX6', 'MBGA', 'MGC', 'MSN', 'NP', 'OGC', 'OPC', 'PER',
        'PRERC', 'PVALB', 'PV_ChCs', 'SIGA', 'SMC', 'SNCG', 'SST',
        'SST_CHODL', 'SUB', 'THMGA', 'VIP'], dtype=object),
 array([ 15981,  59895,  60331,  22625,   8294,  10080,  40051,   6181,
         42303,  61381,    389,   3217,  30659, 106583,  82889,  82424,
         58832, 100150,  89218,  61566,  91144,  62992,  42667,  45312,
          7847,  56709, 105681,  36988,  59358,  64286,     48,  50576,
         74543,  40875,   1968,  16796,  28580,  50321,  22444,   1424,
         10714,  47409]))

In [47]:
df.loc[df['adjusted p-value'] < 0.05].shape

(1861731, 5)

In [46]:
df.loc[df['adjusted p-value'] < 0.05].to_csv("snATAC_matched_MiniAtlas_cCRE.diff.LRtest.csv")

In [30]:
prop_lst = []
for t in np.unique(atac_peaks.obs['subclass']):
    idx = np.where(atac_peaks.obs['subclass'] == t)
    total_frag = np.sum(atac_peaks.obs.iloc[idx]['n_fragment'])
    prop = atac_peaks.X[idx[0], :].sum(axis=0)
    prop = (prop * 10000) / total_frag
    prop_lst.append(prop)
    
prop_df = pd.DataFrame([mat.A1 for mat in prop_lst], index=np.unique(atac_peaks.obs['subclass']), columns = atac_peaks.var_names).T
prop_df.to_csv('snATAC_matched_MiniAtlas_subclass_cCRE_agg.csv')